# Stock Analytic Platform 
## Description : Allow user to backtest their algorithm on the bursa stock model  

##Objective : To learn how algo trading works via building back testing

##Program Flow : 
###1.Retrieve all historical price for every stock on bursa and have it update on a daily basis
###2.Retrieve all comments for each stock 
###3.Plot out all Stock history price on matlibplot 
###4.create the algorithm to testify the correlation between comment count versus historical price [high | low | volume]
###5.Create a web platform from it, and will allow user to customize the logic they are intend to use.
###6.SHIP IT.


In [ ]:
##To download all historical stock price

from bs4 import BeautifulSoup
import urllib
import urllib2
import re
import csv
import IPython.lib.display
import datetime as DT
import os.path


symbols=[]
quotes=[]


def comments_url_get(x):
        
    link = "http://klse.i3investor.com/jsp/scl/forum.jsp?fp=1&c=1" 
    page = urllib2.urlopen(link)
    soup = BeautifulSoup(page.read(),"lxml")
    comment_urls=[]
    i=0
    
    while(soup.body.findAll(**{'class':'forumtopic'})):
         i+=1
         link = "http://klse.i3investor.com/jsp/scl/forum.jsp?fp="+str(i)+"&c=1" 
         page = urllib2.urlopen(link)
         soup = BeautifulSoup(page.read(),"lxml")
         comment_urls.extend(re.sub('fp=\d+','',url['href'].encode('utf-8')) for url in soup.find_all('a',href=re.compile('/servlets/forum/\d+\.jsp'))) 
    print("fetch complete, writing now ...")
    with open ('comment_urls.csv','w') as fp:
        writer = csv.writer(fp,delimiter='\n')
        writer.writerows([comment_urls])
    print("write completed")
        
    
def stock_quote_get(fname):
    with open(fname) as f:
        content = f.readlines()
    for url in content:
        url.replace('\n','')
        link = "http://klse.i3investor.com"+url
        stock_quote_retrieval(link)
        with open('symbol_link.csv', 'w') as fp:
            a = csv.writer(fp,delimiter='\n')
            a.writerows([symbols])
    
def stock_quote_retrieval(link):
    page = urllib2.urlopen(link)
    soup = BeautifulSoup(page.read(),"lxml")
    symbols.append(soup.find('a', href=re.compile('/servlets/stk/.*\.jsp'))['href'])

#stock_quote_get("comment_urls.csv")

def symbol_retrieval(fname):
    with open(fname) as f:
        content = f.readlines()
    for symbol in content:
        quotes.append(re.search('(\d{4}(\w+)?)',symbol).group(1))
    with open('symbol_list.csv','w') as fp:
            writer = csv.writer(fp,delimiter='\n')
            writer.writerows([quotes])
            
#symbol_retrieval("symbol_link.csv")            



def stock_price_get(fname):
    
    save_path="/Users/ryan/DevOps/python/project_2_stock_analytic/symbols/"    
    
    with open('symbol_list.csv') as f:
        content = f.read().splitlines()
    for symbol in content: 
        urllib.urlretrieve("http://real-chart.finance.yahoo.com/table.csv?s="+symbol+".KL&d=7&e=18&f=2015&g=d&a=0&b=3&c=2000&ignore=.csv",os.path.join(save_path,symbol+".csv"))
             
             
                                   
                           
stock_price_get("symbol_list.csv")


In [ ]:
%matplotlib inline
%pylab inline 
import matplotlib.pyplot as p
import matplotlib.cbook as cbook

fname = cbook.get_sample_data('bat_hp.csv', asfileobj=False)
#p.plotfile(fname, (0,5,6))

p.plotfile(fname, ('date', 'open','high','low','close'), subplots=False)

p.show()